## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user google-adk

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [ ]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Serach Agent App の作成

In [1]:
import json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
instruction = '''
You are a friendly AI assistant that answers your queries.
Use google_search to give answers based on the latest and objective information.
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [3]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._app_name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [4]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

query = '''
今日の新宿区の最高予想気温は？
'''
_ = await client.stream(query)

 今日の新宿区の最高予想気温は25℃です。



SessionService が管理するセッション情報を確認します。

セッションを特定するのに必要な、`app_name`、`user_id`、`session_id` を確認します。

In [5]:
client._session.app_name, client._session.user_id, client._session.id

('Search Agent App', 'user00', '9e1100a431cc4c8e83bd810f15c6591b')

確認した情報を使って、セッションに記録されたイベントを表示します。

2 つのイベント（ユーザーの質問とエージェントの回答）が記録されています。

エージェントの回答を生成する際に用いた Google Serch の情報なども確認できます。

In [6]:
client._runner.session_service.get_session(
    app_name = client._session.app_name,
    user_id = client._session.user_id,
    session_id = client._session.id,
).events

[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='\n今日の新宿区の最高予想気温は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, invocation_id='e-e494f5a5-31f8-4608-b282-c46c701d6294', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='2pTsFz1O', timestamp=1747283395.096116),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text=' 今日の新宿区の最高予想気温は25℃です。\n')], role='model'), grounding_metadata=GroundingMetadata(grounding_chunks=[GroundingChunk(retrieved_context=

続けて質問すると、セッション情報を踏まえた回答が得られます。

In [7]:
query = '''
那覇はどうよ？
'''
_ = await client.stream(query)

 今日の那覇市の最高気温は28℃と予想されています。



先ほどと同様にセッションに記録されたイベントを確認すると、新しい質問と回答のイベントが追加されています。

In [8]:
client._runner.session_service.get_session(
    app_name = client._session.app_name,
    user_id = client._session.user_id,
    session_id = client._session.id,
).events

[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='\n今日の新宿区の最高予想気温は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, invocation_id='e-e494f5a5-31f8-4608-b282-c46c701d6294', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='2pTsFz1O', timestamp=1747283395.096116),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text=' 今日の新宿区の最高予想気温は25℃です。\n')], role='model'), grounding_metadata=GroundingMetadata(grounding_chunks=[GroundingChunk(retrieved_context=

## システムインストラクションの確認

ちょっとした裏技を使って、LlmAgent オブジェクトが Gemini API に送っている生の情報を覗いてみます。

In [9]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async2(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
        print('## Tools config ##')
        pprint.pp(llm_request.config.tools)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
BaseLlmFlow._run_one_step_async = _run_one_step_async2

In [10]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
今日の新宿区の最高予想気温は？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='\n今日の新宿区の最高予想気温は？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers your queries.
Use google_search to give answers based on the latest and objective information.


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions using Google Search."
----
## Tools config ##
[Tool(function_declarations=None, retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, code_execution=None)]
----
 今日の新宿区の最高予想気温は25℃です。


In [11]:
query = '''
那覇はどうよ？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='\n今日の新宿区の最高予想気温は？\n')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text=' 今日の新宿区の最高予想気温は25℃です。')], role='model'),
 UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='\n那覇はどうよ？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers your queries.
Use google_search to give answers based on the latest and objective information.


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions usin